In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [43]:
# Working through Question #4 - UrLeaka

### Hi Chris

In [44]:
accre_data = pd.read_csv('~/nss_data_science/DataQuestion2/data-question-2-dream-crushers/data/july2017-june2018.txt', delimiter='|', nrows=100)
accre_data.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [45]:
accre_data.columns

Index(['JobID', 'JobIDRaw', 'Cluster', 'Partition', 'Account', 'Group', 'GID',
       'User', 'UID', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName'],
      dtype='object')

In [46]:
accre_data.Start.head()
# accre_data['RunCalc'] = accre_data['End'] - accre_data['Start']

0    2017-07-03T02:13:59
1    2017-07-03T02:20:08
2    2017-07-03T02:41:13
3    2017-07-03T02:51:53
4    2017-07-01T05:00:01
Name: Start, dtype: object

In [47]:
accre_data.Start = pd.to_datetime(accre_data.Start)
accre_data.End = pd.to_datetime(accre_data.End)

In [48]:
accre_data.Elapsed.head()

0    1-01:05:21
1      23:51:47
2    1-01:59:51
3    1-08:02:20
4      00:00:10
Name: Elapsed, dtype: object

In [49]:
accre_data['RunCalc'] = accre_data.End - accre_data.Start

In [50]:
accre_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 26 columns):
JobID        100 non-null object
JobIDRaw     100 non-null int64
Cluster      100 non-null object
Partition    100 non-null object
Account      100 non-null object
Group        100 non-null object
GID          100 non-null int64
User         100 non-null object
UID          69 non-null float64
Submit       100 non-null object
Eligible     100 non-null object
Start        100 non-null datetime64[ns]
End          100 non-null datetime64[ns]
Elapsed      100 non-null object
ExitCode     100 non-null object
State        100 non-null object
NNodes       100 non-null int64
NCPUS        100 non-null int64
ReqCPUS      100 non-null int64
ReqMem       100 non-null object
ReqGRES      0 non-null float64
ReqTRES      100 non-null object
Timelimit    100 non-null object
NodeList     100 non-null object
JobName      100 non-null object
RunCalc      100 non-null timedelta64[ns]
dtypes: datetime64[

In [51]:
accre_data.RunCalc.head()

0   1 days 01:05:21
1   0 days 23:51:47
2   1 days 01:59:51
3   1 days 08:02:20
4   0 days 00:00:10
Name: RunCalc, dtype: timedelta64[ns]

In [52]:
days=accre_data.RunCalc.dt.days
days.head()

0    1
1    0
2    1
3    1
4    0
Name: RunCalc, dtype: int64

In [118]:
completed=accre_data[accre_data.State == 'COMPLETED']
completed['days']=days
less24hrs = completed[completed.days< 1]
less24hrs.head()

C:\Users\unews\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,RunCalc,days
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,23:51:47,0
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,00:00:10,0
5,15811618,15811618,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:00:01,...,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1063,skim_test,00:03:18,0
6,15811619,15811619,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-01T05:00:02,...,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1062,skim_test,00:02:03,0
7,15811621,15811621,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:22,...,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_33ee450ebdc4,00:00:10,0


In [119]:
less24hrs.days.head()

1    0
4    0
5    0
6    0
7    0
Name: days, dtype: int64

In [120]:
less24hrs.RunCalc.head()

1   23:51:47
4   00:00:10
5   00:03:18
6   00:02:03
7   00:00:10
Name: RunCalc, dtype: timedelta64[ns]

In [121]:
secs_RunCalc=less24hrs.RunCalc.dt.seconds
less24hrs['secs_RunCalc'] = secs_RunCalc
#Number of seconds (>= 0 and less than 1 day) for each element.

C:\Users\unews\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [122]:
less24hrs.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,RunCalc,days,secs_RunCalc
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,23:51:47,0,85907
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,00:00:10,0,10
5,15811618,15811618,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:00:01,...,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1063,skim_test,00:03:18,0,198
6,15811619,15811619,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-01T05:00:02,...,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1062,skim_test,00:02:03,0,123
7,15811621,15811621,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:22,...,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_33ee450ebdc4,00:00:10,0,10


In [64]:
accre_data.loc[4]

JobID                      15811617
JobIDRaw                   15811617
Cluster                       accre
Partition                production
Account                 cms_samtest
Group                   cms_samtest
GID                           59297
User                       uscms010
UID                             NaN
Submit          2017-07-01T05:00:00
Eligible        2017-07-01T05:00:00
Start           2017-07-01 05:00:01
End             2017-07-01 05:00:11
Elapsed                    00:00:10
ExitCode                        0:0
State                     COMPLETED
NNodes                            1
NCPUS                             1
ReqCPUS                           1
ReqMem                       4000Mn
ReqGRES                         NaN
ReqTRES      cpu=1,mem=4000M,node=1
Timelimit                2-00:00:00
NodeList                    vmp1242
JobName             bl_aa7c22e04f21
RunCalc             0 days 00:00:10
days                              0
Name: 4, dtype: object

In [123]:
secs_RunCalc300s = less24hrs[less24hrs.secs_RunCalc<=300]
secs_RunCalc300s.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,RunCalc,days,secs_RunCalc
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,00:00:10,0,10
5,15811618,15811618,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:00:01,...,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1063,skim_test,00:03:18,0,198
6,15811619,15811619,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-01T05:00:02,...,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1062,skim_test,00:02:03,0,123
7,15811621,15811621,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:22,...,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_33ee450ebdc4,00:00:10,0,10
8,15811624,15811624,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:01:01,...,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp670,skim_test,00:04:47,0,287


In [66]:
secs_RunCalc300s.Elapsed.head()

4    00:00:10
5    00:03:18
6    00:02:03
7    00:00:10
8    00:04:47
Name: Elapsed, dtype: object

In [124]:
#Name Change
prob_jobs=secs_RunCalc300s

In [125]:
prob_jobs.shape

(74, 28)

In [ ]:
#Out of 100 cases selected in intial download, 74 cases may be problematic due to short run time.  Narrow down jobs within 4 hours later...

In [126]:
prob_users=prob_jobs.User.value_counts()
print(prob_users)

nagarr1     63
uscms010     3
autocms      3
tuos         3
vuiiscci     2
Name: User, dtype: int64


In [102]:
# prob_user_df=prob_jobs.loc[prob_jobs['User'].isin(prob_users)] -- did not work when setting prob users >20 or 50

In [131]:
pu1=prob_users>50
pu1.head()

nagarr1      True
uscms010    False
autocms     False
tuos        False
vuiiscci    False
Name: User, dtype: bool

In [158]:
pu2=pu1.to_dict()#changes series to dictionary where each user is associated with a boolean factor
type(pu2)

dict

In [185]:
prob_jobs['greater500']=prob_jobs['User'].map(pu2)#code "Pandas’ map function lets you add a new column 
# with values from a dictionary if the data frame has a column matching the keys in the dictionary."
# http://cmdlinetips.com/2018/01/how-to-add-a-new-column-to-using-a-dictionary-in-pandas-data-frame/

C:\Users\unews\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [178]:
prob_jobs.shape

(74, 29)

In [186]:
prob_jobs['greater500']

4     False
5     False
6     False
7     False
8     False
9     False
31    False
33    False
34    False
35    False
36    False
37     True
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49     True
50     True
51     True
52     True
53     True
54     True
55     True
      ...  
70     True
71     True
72     True
73     True
74     True
75     True
76     True
77     True
78     True
79     True
80     True
81     True
82     True
83     True
84     True
85     True
86     True
87     True
88     True
89     True
90     True
91     True
92     True
93     True
94     True
95     True
96     True
97     True
98     True
99     True
Name: greater500, Length: 74, dtype: bool

In [191]:
# prob_users1=prob_jobs[prob_jobs.greater500 =='True'] - will not filter columns because column is not string but bool
type(prob_jobs.greater500)

pandas.core.series.Series

In [194]:
type(prob_jobs.iloc[4,29])

numpy.bool_

In [196]:
np.dtype(prob_jobs.greater500)

dtype('bool')

In [197]:
prob_jobs['greater500'] = prob_jobs['greater500'].astype(str)

C:\Users\unews\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [198]:
np.dtype(prob_jobs.greater500) #converted column to str in order to select for true

dtype('O')

In [199]:
prob_jobs.greater500.head()

4    False
5    False
6    False
7    False
8    False
Name: greater500, dtype: object

In [200]:
prob_users1=prob_jobs[prob_jobs.greater500 =='True']

In [201]:
prob_users1.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,RunCalc,days,secs_RunCalc,>500,greater500
37,15811657_261,15811657,accre,production,p_iglab_csb,p_iglab,20440,nagarr1,497968.0,2017-07-01T05:02:05,...,NaN,"cpu=1,mem=15G,node=1",1-00:00:00,vmp1331,fp_single.slurm,00:00:09,0,9,True,True
38,15811657_1,15811659,accre,production,p_iglab_csb,p_iglab,20440,nagarr1,497968.0,2017-07-01T05:02:05,...,NaN,"cpu=1,mem=15G,node=1",1-00:00:00,vmp202,fp_single.slurm,00:00:08,0,8,True,True
39,15811657_2,15811660,accre,production,p_iglab_csb,p_iglab,20440,nagarr1,497968.0,2017-07-01T05:02:05,...,NaN,"cpu=1,mem=15G,node=1",1-00:00:00,vmp203,fp_single.slurm,00:00:19,0,19,True,True
40,15811657_3,15811661,accre,production,p_iglab_csb,p_iglab,20440,nagarr1,497968.0,2017-07-01T05:02:05,...,NaN,"cpu=1,mem=15G,node=1",1-00:00:00,vmp203,fp_single.slurm,00:00:17,0,17,True,True
41,15811657_4,15811662,accre,production,p_iglab_csb,p_iglab,20440,nagarr1,497968.0,2017-07-01T05:02:05,...,NaN,"cpu=1,mem=15G,node=1",1-00:00:00,vmp1091,fp_single.slurm,00:00:20,0,20,True,True


In [202]:
# accre_data.Start = pd.to_datetime(accre_data.Start)
prob_users1.Submit = pd.to_datetime(prob_users1.Submit)

C:\Users\unews\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [203]:
prob_users1.Submit.head()

37   2017-07-01 05:02:05
38   2017-07-01 05:02:05
39   2017-07-01 05:02:05
40   2017-07-01 05:02:05
41   2017-07-01 05:02:05
Name: Submit, dtype: datetime64[ns]

In [207]:
prob_users2=prob_users1.sort_values(['User','Submit'],ascending=[1,1])

In [224]:
pu4hrs['count_num']=prob_users2.rolling('14400s', on='Submit').JobID.count()

In [225]:
print(pu4hrs.count_num)

37     1.0
38     2.0
39     3.0
40     4.0
41     5.0
42     6.0
43     7.0
44     8.0
45     9.0
46    10.0
47    11.0
48    12.0
49    13.0
50    14.0
51    15.0
52    16.0
53    17.0
54    18.0
55    19.0
56    20.0
57    21.0
58    22.0
59    23.0
60    24.0
61    25.0
62    26.0
63    27.0
64    28.0
65    29.0
66    30.0
      ... 
70    34.0
71    35.0
72    36.0
73    37.0
74    38.0
75    39.0
76    40.0
77    41.0
78    42.0
79    43.0
80    44.0
81    45.0
82    46.0
83    47.0
84    48.0
85    49.0
86    50.0
87    51.0
88    52.0
89    53.0
90    54.0
91    55.0
92    56.0
93    57.0
94    58.0
95    59.0
96    60.0
97    61.0
98    62.0
99    63.0
Name: JobID, Length: 63, dtype: float64


In [226]:
# secs_RunCalc300s = less24hrs[less24hrs.secs_RunCalc<=300]
prob_users3=pu4hrs[pu4hrs.count_num>50]#change later -- looking for counts greater than 500.... then select for users.

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match